In [4]:
from datetime import datetime
import json

class Producto:
    def __init__(self, id, nombre, fecha_vencimiento=None, perecible=False, cantidad=0):
        self.id = id
        self.nombre = nombre
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.perecible = perecible
        self.cantidad = cantidad

    def __str__(self):
        fecha = self.fecha_vencimiento.strftime("%Y-%m-%d") if self.fecha_vencimiento else "No vence"
        return f"{self.nombre} (ID: {self.id}, Cantidad: {self.cantidad}, Vence: {fecha})"

class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo
        self.productos = []
        self.hijos = []

    def agregar_producto(self, producto):
        self.productos.append(producto)
        if producto.perecible:
            self.productos.sort(key=lambda x: x.fecha_vencimiento if x.fecha_vencimiento else datetime.max)

    def agregar_hijo(self, nodo):
        self.hijos.append(nodo)

    def mostrar_arbol(self, nivel=0):
        print("  " * nivel + f"- {self.tipo}")
        for producto in self.productos:
            print("  " * (nivel + 1) + f"* {producto}")
        for hijo in self.hijos:
            hijo.mostrar_arbol(nivel + 1)

def cargar_productos_desde_archivo(inventario):
    try:
        # Crear el árbol principal
        arbol_productos = Nodo("Almacén")
        
        # Crear nodos para las categorías
        perecederos = Nodo("Perecederos")
        no_perecederos = Nodo("No Perecederos")
        arbol_productos.agregar_hijo(perecederos)
        arbol_productos.agregar_hijo(no_perecederos)

        # Leer el archivo línea por línea
        with open(inventario, 'r', encoding='utf-8') as file:
            for linea in file:
                try:
                    # Convertir cada línea en un diccionario
                    prod = json.loads(linea.strip())
                    
                    # Crear producto
                    producto = Producto(
                        id=prod["id"],
                        nombre=prod["nombre"],
                        perecible=prod["tipo"] == "perecedero",
                        cantidad=prod["cantidad"],
                        fecha_vencimiento=prod['fech_venc'],
                    )
                    
                    # Agregar al nodo correspondiente
                    if prod["tipo"] == "perecedero":
                        perecederos.agregar_producto(producto)
                    else:
                        no_perecederos.agregar_producto(producto)
                        
                except json.JSONDecodeError as e:
                    print(f"Error al decodificar línea: {linea.strip()}")
                    print(f"Error: {e}")
                    continue
                    
        return arbol_productos
        
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{inventario}'")
        return None
    except Exception as e:
        print(f"Error inesperado: {e}")
        return None

# Ejemplo de uso
if __name__ == "__main__":
    # Cargar el árbol desde el archivo
    arbol = cargar_productos_desde_archivo('inventario.txt')
    
    if arbol:
        print("\nÁrbol de productos cargado:")
        arbol.mostrar_arbol()



Árbol de productos cargado:
- Almacén
  - Perecederos
    * Manzana (ID: 1, Cantidad: 50, Vence: 2024-10-26)
    * Pollo (ID: 5, Cantidad: 20, Vence: 2024-10-27)
    * Leche (ID: 3, Cantidad: 30, Vence: 2025-11-10)
  - No Perecederos
    * Arroz (ID: 2, Cantidad: 100, Vence: 2030-01-20)
    * Lentejas (ID: 4, Cantidad: 80, Vence: 2024-10-26)
    * Pasta (ID: 6, Cantidad: 60, Vence: 2027-12-12)
